In [1]:
import streamlit as st
import pandas as pd
import joblib
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

vct = joblib.load('tfidf_vectorizer.pkl')

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_excel('.\\Data\\final_df.xlsx')
df

,FullText,hoax
0,mantan gubernur dki jakarta anies baswedan had...,0
1,gubernur sumatera utara edy rahmayadi buka mun...,0
2,partai bangkit bangsa pkb bakal usung menteri ...,0
3,ketua umum partai bangkit nusantara pkn i gede...,0
4,dewan pimpin pusat dpp partai bangkit nusantar...,0
...,...,...
31303,foto raja salman angkat tandu korban crane tid...,1
31304,trik halftruth tampak jokowi sedang hormat tid...,1
31305,proyek pltu sebut benar jalan sejak tahun tahu...,1
31306,mata uang indonesia puruk dolar amerika lah ma...,1


In [3]:
X = df['FullText']
y = df['hoax']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
import pandas as pd
import joblib

# Transform the data
X_train_vector = vct.fit_transform(X_train)



X_test_vector = vct.transform(X_test)


In [5]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

model.fit(X_train_vector, y_train)

RandomForestClassifier()

In [6]:
from sklearn.metrics import classification_report, accuracy_score

# Memprediksi label untuk data pengujian
y_pred = model.predict(X_test_vector)

# Evaluasi kinerja model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)


In [7]:
accuracy

0.9763653784733312

In [8]:
print(report)

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      4221
           1       0.99      0.93      0.96      2041

    accuracy                           0.98      6262
   macro avg       0.98      0.97      0.97      6262
weighted avg       0.98      0.98      0.98      6262



In [10]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# Menghitung Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Menampilkan Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=model.classes_, yticklabels=model.classes_)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

C:\Users\Bintang\AppData\Local\Temp\ipykernel_23772\3927385783.py:14: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

# Parameter grid untuk GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Inisialisasi RandomForestClassifier
rf = RandomForestClassifier(random_state=42)

# Inisialisasi GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Melakukan fitting pada data pelatihan
grid_search.fit(X_train_vector, y_train)

# Menampilkan hasil terbaik dari GridSearchCV
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Estimator: {grid_search.best_estimator_}")

# Menggunakan model terbaik untuk melakukan prediksi pada data pengujian
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test_vector)

# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')

Fitting 5 folds for each of 648 candidates, totalling 3240 fits


c:\Users\Bintang\OneDrive\Desktop\Bintang\3IA01\PI\Project\pi-venv\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1080 fits failed out of a total of 3240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
854 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Bintang\OneDrive\Desktop\Bintang\3IA01\PI\Project\pi-venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Bintang\OneDrive\Desktop\Bintang\3IA01\PI\Project\pi-venv\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "c:\Users\Bintang\OneDrive\Desktop\Bintang\3IA01

Best Parameters: {'bootstrap': False, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best Estimator: RandomForestClassifier(bootstrap=False, n_estimators=300, random_state=42)
Accuracy: 0.978122005748962
Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      4221
           1       0.99      0.94      0.97      2041

    accuracy                           0.98      6262
   macro avg       0.98      0.97      0.97      6262
weighted avg       0.98      0.98      0.98      6262

